In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
!pip install -q segmentation_models_pytorch
import segmentation_models_pytorch as smp
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.checkpoint as C
import torchvision.transforms.functional as fn
import torchvision.transforms as T
import matplotlib.pyplot as plt
!pip install -q torchsummary
from torchvision import models
from torchsummary import summary

In [11]:
train_path = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/train'

In [12]:
ids = os.listdir(train_path)

convert images to ash colour fromat 

In [6]:
_T11_BOUNDS = (243, 303)
_CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
_TDIFF_BOUNDS = (-4, 2)

def normalize_range(data, bounds):
    """Maps data to the range [0, 1]."""
    return (data - bounds[0]) / (bounds[1] - bounds[0])

In [9]:
os.mkdir('/kaggle/working/data_dir')

In [ ]:
for i in ids:
    band15= np.load(f'{train_path}/{i}/band_15.npy')
    band14= np.load(f'{train_path}/{i}/band_14.npy')
    band11= np.load(f'{train_path}/{i}/band_11.npy')
    r= normalize_range(band15 - band14, _TDIFF_BOUNDS)
    g= normalize_range(band14 - band11, _CLOUD_TOP_TDIFF_BOUNDS)
    b= normalize_range(band14, _T11_BOUNDS)
    image = np.clip((np.stack([r,g,b],axis=2)),0,1)
    image = image[...,4]
    mask=np.load(f'{train_path}/{i}/human_pixel_masks.npy')
#     os.mkdir(f'/kaggel/data_dir/{i}')
    directory = f'/kaggle/working/data_dir/{i}/'
    os.makedirs(directory, exist_ok=True)
    np.save(os.path.join(directory, 'false.npy'), image)
    np.save(os.path.join(directory, 'mask.npy'), mask)

In [ ]:
test_image = np.load(f'/kaggle/working/data_dir/{tmp[156]}/false.npy')
test_mask = np.load(f'/kaggle/working/data_dir/{tmp[156]}/mask.npy')
print(test_image.shape)
print(test_mask.shape)

In [ ]:
fig , ax = plt.subplots(1,2)
ax[0].imshow(test_image)
ax[1].imshow(test_mask)

custom model input 256,256,3 -> output 256,256

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,path):
        self.path = path
        self.ids = os.listdir(self.path)
    def __len__(self):
        return len(self.ids)
    def __getitem__(self,idx):
        id = ids[idx]
        image = np.load(f'{self.path}/{id}/false.npy')
        mask = np.load(f'{self.path}/{id}/mask.npy')
        image = torch.from_numpy(image)
        image = torch.moveaxis(image,-1,0)
        mask = torch.from_numpy(mask)
        mask=torch.moveaxis(mask,-1,0)
        return image,mask.float()

In [ ]:
train_dataset = CustomDataset('/kaggle/working/data_dir')

In [ ]:
train_dataloader = DataLoader(train_dataset,batch_size=64, shuffle=True)

In [ ]:
test_image , test_mask = next(iter(train_dataloader))
test_image = torch.moveaxis(test_image,1,-1)
test_mask = torch.moveaxis(test_mask,1,-1)
ax , fig = plt.subplots(1,2)
fig[0].imshow(test_image[2])
fig[1].imshow(test_mask[2])

In [ ]:
model = smp.Unet(encoder_name='resnet34',encoder_weights='imagenet',in_channels=3,classes=1)

In [ ]:
summary(model,(3,256,256))

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def forward(self, predicted, target):
        smooth = 1e-6  # Smoothing factor to avoid division by zero
        predicted_flat = predicted.view(-1)
        target_flat = target.view(-1)
        
        intersection = torch.sum(predicted_flat * target_flat)
        union = torch.sum(predicted_flat) + torch.sum(target_flat)
        
        dice_coefficient = (2.0 * intersection + smooth) / (union + smooth)
        dice_loss = 1.0 - dice_coefficient
        
        return dice_loss

    

In [ ]:
class args:
    epochs:40

In [ ]:
os.mkdir('/kaggel/working/savedmodel')

In [ ]:
def train(args):
    model.train()
    learning_rate = 0.005
    losses = []
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',patience = 4, factor = 0.31622776601, verbose = True)
    best_score = 0
    best_dice_epoch = 0
    for epoch in range(args.epochs):
        batch_dice_score = 0
        net_dice_loss = 0
        if(epoch - best_dice_epoch > 10):
            print(f'early stoped due to no improvement in last 10 epochs')
            torch.save(model.state_dict(),f'/kaggel/working/savedmodel/{epoch}th_epoch.pt')
            break
        for image , mask in tqdm(train_dataloader):
            optimizer.zero_grad()
            pred = model(image)
            loss = Diceloss.forward(pred,mask)
            loss.backward()
            optimizer.step()
            batch_dice_score += (1-loss)
            net_dice_loss +=loss
        losses.append(net_dice_loss)
        print(f'at {epoch}th the dice loss is {net_dice_loss}')
        if(best_score<batch_dice_score):
            torch.save(model.state_dict(),f'/kaggel/working/savedmodel/{epoch}th_epoch.pt')
            torch.save(model,'{epoch}th_model.pt')
            best_score = batch_dice_score
            best_dice_epoch = epoch
            